# Objective
- Create a Convolutional Neural Network for classifying nuts and bots
- To learn the use of commonly used deep learning layers

# Import modules

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,\
Conv2D, MaxPooling2D, Permute, Reshape
import skimage
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from skimage.io import imread
from skimage.transform import resize
import keras
from keras import backend as K
import tensorflow
#K.set_session('sess')

Using TensorFlow backend.


# Declare paths for data

In [2]:
train_path = '../data/nuts_n_bolts_master/all_dl/train'
test_path = '../data/nuts_n_bolts_master/all_dl/test'
val_path = '../data/nuts_n_bolts_master/all_dl/val'

# Load data for training and testing

In [3]:
batch_size = 25
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
        train_path,  # this is the target directory
        target_size=(50, 65),  # all images will be resized to 150x150
        batch_size=batch_size,
        classes = ['nuts','bolts'],
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels


Found 104 images belonging to 2 classes.


In [4]:
batch_size = 4
val_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
val_generator = val_datagen.flow_from_directory(
        val_path,  # this is the target directory
        target_size=(50, 65),  # all images will be resized to 50x65
        batch_size=batch_size,
        classes = ['nuts','bolts'],
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

Found 30 images belonging to 2 classes.


In [5]:
batch_size = 10
test_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_generator = test_datagen.flow_from_directory(
        test_path,  # this is the target directory
        target_size=(50, 65),  # all images will be resized to 50x65
        batch_size=batch_size,
        classes = ['nuts','bolts'],
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

Found 57 images belonging to 2 classes.


# Construct a simple CNN model for classification

In [6]:
# Construct a model in keras for nuts_and_bolts classfication
# The model consists of two convolutional layers followed by Relu
model = Sequential()

Declare 2 convolutional layers for the model

In [7]:
model.add(Conv2D(32, (3, 3), input_shape=(50,65,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))

Declare rest of the network for the model
- Add a maxpooling layer
- Add two fully connected layer, the final fully connected layer will be the classifier output

In [8]:
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Flatten())
a,b,c,d = model.output_shape
a = b * c * d
model.add(Permute((1, 2, 3)))  # Indicate NHWC data layout
model.add(Reshape((a,)))
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [9]:
model.fit_generator(
        train_generator,
        steps_per_epoch=100 // batch_size,
        epochs=5,
        validation_data=val_generator,
        validation_steps=100 // batch_size)

Epoch 1/5
10/10 [==============================] - 4s 423ms/step - loss: 1.1023 - acc: 0.5639 - val_loss: 0.5864 - val_acc: 0.6316
Epoch 2/5
10/10 [==============================] - 4s 365ms/step - loss: 0.5544 - acc: 0.7284 - val_loss: 0.4016 - val_acc: 0.8421
Epoch 3/5
10/10 [==============================] - 4s 353ms/step - loss: 0.3951 - acc: 0.8315 - val_loss: 0.1913 - val_acc: 0.9211
Epoch 4/5
10/10 [==============================] - 4s 368ms/step - loss: 0.3107 - acc: 0.8587 - val_loss: 0.2001 - val_acc: 0.9474
Epoch 5/5
10/10 [==============================] - 4s 350ms/step - loss: 0.3335 - acc: 0.8697 - val_loss: 0.2549 - val_acc: 0.9474


# Test loss function and accuracy

In [10]:
model.summary()
score = model.evaluate_generator(test_generator, steps=50)
print('Test score:', score[0])
print('Test accuracy:', score[1])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 63, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 30, 32)        0         
_________________________________________________________________
permute_1 (Permute)          (None, 23, 30, 32)        0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 22080)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               2826368   
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total para

In [11]:
# serialize model to JSON

sess = tensorflow.keras.backend.get_session()
constant_graph = tf.graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), ['activation_4/Softmax'])
tf.train.write_graph(constant_graph, "", "graph.pb", as_text=False)

model_json = model.to_json()
with open("../models/nuts_and_bolts_dl.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("../models/nuts_and_bolts_dl.h5")
print("Saved model to disk")

AttributeError: 'module' object has no attribute 'keras'

# Now let's increase the number of convolutional layers

In [11]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(50,65,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

# Lets test results for our new model

In [12]:
model.fit_generator(
        train_generator,
        steps_per_epoch=100 // batch_size,
        epochs=5,
        validation_data=val_generator,
        validation_steps=100 // batch_size)

Epoch 1/5
10/10 [==============================] - 9s 876ms/step - loss: 1.2440 - acc: 0.4740 - val_loss: 0.6934 - val_acc: 0.4737
Epoch 2/5
10/10 [==============================] - 8s 802ms/step - loss: 0.6789 - acc: 0.6316 - val_loss: 0.6263 - val_acc: 0.9211
Epoch 3/5
10/10 [==============================] - 8s 792ms/step - loss: 0.6090 - acc: 0.7419 - val_loss: 0.4348 - val_acc: 0.8684
Epoch 4/5
10/10 [==============================] - 8s 802ms/step - loss: 0.4239 - acc: 0.8261 - val_loss: 0.2337 - val_acc: 0.8421
Epoch 5/5
10/10 [==============================] - 8s 800ms/step - loss: 0.2777 - acc: 0.8831 - val_loss: 0.2063 - val_acc: 0.8684


# Test loss function and accuracy

In [14]:
score = model.evaluate_generator(test_generator, steps=50)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.512016474402
Test accuracy: 0.815126048918


# Lets analyse the effect of adding a few drop out layers

In [15]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(50,65,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

# Train the model to analyse if dropout increases accuracy

In [16]:
model.fit_generator(
        train_generator,
        steps_per_epoch=100 // batch_size,
        epochs=5,
        validation_data=val_generator,
        validation_steps=100 // batch_size)

Epoch 1/5
10/10 [==============================] - 9s 892ms/step - loss: 0.7307 - acc: 0.4903 - val_loss: 0.6795 - val_acc: 0.4737
Epoch 2/5
10/10 [==============================] - 8s 809ms/step - loss: 0.5810 - acc: 0.6889 - val_loss: 0.3036 - val_acc: 0.9211
Epoch 3/5
10/10 [==============================] - 8s 832ms/step - loss: 0.9397 - acc: 0.6739 - val_loss: 0.4549 - val_acc: 0.7895
Epoch 4/5
10/10 [==============================] - 8s 828ms/step - loss: 0.4931 - acc: 0.7635 - val_loss: 0.4046 - val_acc: 0.8158
Epoch 5/5
10/10 [==============================] - 8s 833ms/step - loss: 0.3777 - acc: 0.8261 - val_loss: 0.2686 - val_acc: 0.8947


# Test loss function and accuracy

In [17]:
score = model.evaluate_generator(test_generator, steps=50)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.570904813112
Test accuracy: 0.766806722814


In [18]:
# Save the trained model
model_json = model.to_json()
with open("../models/nuts_and_bolts_dl.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("../models/nuts_and_bolts.h5")
print("Saved model to disk")

Saved model to disk


# Let's try out different activation functions

In [17]:
activations = ['relu','tanh','sigmoid']


# Define a function to construct and evaluate a model to make the comparison of different loss functions easier

In [18]:
def build_and_evaluate_model(actv):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(50,65,3),activation= actv))
    model.add(Conv2D(32,(3,3),activation= actv))
    model.add(Conv2D(32,(3,3),activation= actv))
    model.add(Conv2D(32,(3,3),activation= actv))
    model.add(Conv2D(32,(3,3),activation= actv))
    model.add(Conv2D(32,(3,3),activation= actv))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(128, activation= actv))
    model.add(Dense(2, activation= actv))

    model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
    model.fit_generator(
        train_generator,
        steps_per_epoch=100 // batch_size,
        epochs=2,
        validation_data=val_generator,
        validation_steps=100 // batch_size)
    score = model.evaluate_generator(test_generator, steps=50)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [20]:
for actv in activations:
    print('ACTIVATION',actv,'\n')
    %timeit -n1 -r1 build_and_evaluate_model(actv)
    print('\n')

ACTIVATION relu 

Epoch 1/2
10/10 [==============================] - 9s 881ms/step - loss: 1.3191 - acc: 0.5517 - val_loss: 0.6931 - val_acc: 0.4737
Epoch 2/2
10/10 [==============================] - 8s 801ms/step - loss: 0.6931 - acc: 0.5216 - val_loss: 0.6931 - val_acc: 0.5263
Test score: 0.693147182465
Test accuracy: 0.550420175612
1 loop, best of 1: 23.3 s per loop


ACTIVATION tanh 

Epoch 1/2
10/10 [==============================] - 9s 884ms/step - loss: 1.0280 - acc: 0.5639 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 2/2
10/10 [==============================] - 8s 801ms/step - loss: 0.6931 - acc: 0.5407 - val_loss: 0.6931 - val_acc: 0.4474
Test score: 0.693147182465
Test accuracy: 0.544117653946
1 loop, best of 1: 23.3 s per loop


ACTIVATION sigmoid 

Epoch 1/2
10/10 [==============================] - 9s 884ms/step - loss: 1.9768 - acc: 0.4361 - val_loss: 1.8768 - val_acc: 0.4474
Epoch 2/2
10/10 [==============================] - 8s 805ms/step - loss: 1.7619 - acc: 0.4211 - val_

In [21]:
bolt_path = '../data/nuts_n_bolts_master/all_dl/test/bolts/'
nut_path = '../data/nuts_n_bolts_master/all_dl/test/nuts'
image_paths = [os.path.join(bolt_path, img_name) for img_name in os.listdir(bolt_path)]
image_paths += [os.path.join(nut_path, img_name) for img_name in os.listdir(nut_path)]
test_images = np.zeros((len(image_paths),50,65,3))

for ind,image_name in enumerate(image_paths):
    test_images[ind, :, :, :] = resize(imread(image_name), (50,65,3))
    
prediction = model.predict(test_images)
labels = ['nuts' if img_pred[0] > img_pred[1] else 'bolts' for img_pred in prediction]
image_names = [os.path.basename(path) for path in image_paths]

/home/senthil/.virtualenvs/sc/lib/python3.4/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


OSError: cannot identify image file <_io.BufferedReader name='../data/nuts_n_bolts_master/all_dl/test/bolts/.DS_Store'>

# Display all nuts image

In [ ]:
for pred in labels:
  if pred == 'nuts':
    plt.figure()
    plt.imshow(test_images[ind,:,:,:])

# Display all bolt images

In [ ]:
  if pred == 'bolts':
    plt.figure()
    plt.imshow(test_images[ind,:,:,:])